In [1]:
import pandas as pd
import numpy as np

# 한국어 NER data : https://github.com/machinereading/KoreanNERCorpus ==> train.txt, 

# 데이터 전처리 및 문장화 

In [2]:
data_sents = []
data = []
sent_no = 0
for line in open('data-ko/train.txt', 'r', encoding = 'utf8'):
    if line[0] == ';' :
        data_sents.append(line[1:].split('\n')[0])
        sent_no += 1
    elif line[0] != '$':  
        row = line.split()
        if len(row) :
            row[0] = 'Sentence: ' + str(sent_no) # sentence number 보정해주기 위한 값
            data.append(row)

In [3]:
len(data_sents)

3555

In [4]:
data_sents[0]

' 한편, AFC챔피언스리그 E조에 속한 포항 역시 대회 8강 진출이 불투명하다 .'

In [5]:
data[0]

['Sentence: 1', '한편', 'NNG', 'O']

In [6]:
len(data)

121009

In [7]:
data = pd.DataFrame(data)

In [8]:
data.columns = ['Sentence #', 'Word', 'POS', 'Tag']

In [9]:
data.head(10)

,Sentence #,Word,POS,Tag
0,Sentence: 1,한편,NNG,O
1,Sentence: 1,",",SP,O
2,Sentence: 1,AFC,SL,O
3,Sentence: 1,챔피언스,NNG,O
4,Sentence: 1,리그,NNG,O
5,Sentence: 1,E,SL,B_OG
6,Sentence: 1,조,NNG,I
7,Sentence: 1,에,JKB,O
8,Sentence: 1,속하,VV,O
9,Sentence: 1,ㄴ,ETM,O


In [10]:
data.tail(10)

,Sentence #,Word,POS,Tag
120999,Sentence: 3555,3,SN,O
121000,Sentence: 3555,.,SF,O
121001,Sentence: 3555,85,SN,O
121002,Sentence: 3555,%,SW,O
121003,Sentence: 3555,씩,XSN,O
121004,Sentence: 3555,하락,NNG,O
121005,Sentence: 3555,하,XSV,O
121006,Sentence: 3555,았,EP,O
121007,Sentence: 3555,다,EF,O
121008,Sentence: 3555,.,SF,O


In [11]:
words = list(set(data['Word'].values))
n_words = len(words)
n_words

13859

In [12]:
import sys

class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            print(sys.exc_info())
            return None


In [13]:
getter = SentenceGetter(data)

In [14]:
sent = getter.get_next()

In [15]:
print(sent)

[('한편', 'NNG', 'O'), (',', 'SP', 'O'), ('AFC', 'SL', 'O'), ('챔피언스', 'NNG', 'O'), ('리그', 'NNG', 'O'), ('E', 'SL', 'B_OG'), ('조', 'NNG', 'I'), ('에', 'JKB', 'O'), ('속하', 'VV', 'O'), ('ㄴ', 'ETM', 'O'), ('포항', 'NNP', 'O'), ('역시', 'MAJ', 'O'), ('대회', 'NNG', 'O'), ('8강', 'NNG', 'O'), ('진출', 'NNG', 'O'), ('이', 'JKS', 'O'), ('불투명', 'NNG', 'O'), ('하', 'VV', 'O'), ('다', 'EC', 'O'), ('.', 'SF', 'O')]


In [16]:
sentences = getter.sentences

In [17]:
sentences[50]

[('비', 'NNG', 'O'),
 ('가', 'JKS', 'O'),
 ('오', 'VV', 'O'),
 ('ㄹ', 'ETM', 'O'),
 ('경우', 'NNG', 'O'),
 ('당일', 'NNG', 'B_DT'),
 ('아침', 'NNG', 'B_TI'),
 ('7', 'SN', 'I'),
 ('시', 'NNB', 'I'),
 ('현재', 'NNG', 'O')]

# CRF training을 위해 단어를 features

In [173]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = {
        'bias': 1.0,
        'word': word,
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word': word1,
            '-1:word.isdigit()': word1.isdigit(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word': word1,
            '+1:word.isdigit()': word1.isdigit(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [174]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [175]:
print(len(X))
print(len(y))
print(len(sentences))

3555
3555
3555


In [176]:
print(len(X[10]))
print(len(y[10]))
print(sentences[10])
print(y[10])
print(X[10])

41
41
[('자연', 'NNG', 'O'), ('휴양림', 'NNG', 'O'), ('의', 'JKG', 'O'), ('숲속', 'NNG', 'O'), ('의', 'JKG', 'O'), ('집', 'NNG', 'O'), ('에서', 'JKB', 'O'), ('찻길', 'NNG', 'O'), ('을', 'JKO', 'O'), ('따르', 'VV', 'O'), ('아', 'EC', 'O'), ('오르', 'VV', 'O'), ('다', 'EC', 'O'), ('오른쪽', 'NNG', 'O'), ('숲속', 'NNG', 'O'), ('의', 'JKG', 'O'), ('정자각', 'NNG', 'O'), ('바로', 'MAG', 'O'), ('옆', 'NNG', 'O'), ('계곡', 'NNG', 'O'), ('으로', 'JKB', 'O'), ('내려가', 'VV', 'O'), ('면', 'EC', 'O'), ('적가리', 'NNP', 'B_LC'), ('골', 'NNG', 'I'), ('최고', 'NNG', 'O'), ('의', 'JKG', 'O'), ('경관', 'NNG', 'O'), ('지', 'NNG', 'O'), ('이', 'VCP', 'O'), ('ㄴ', 'ETM', 'O'), ('이', 'NNP', 'B_LC'), ('폭포', 'NNG', 'I'), ('와', 'JC', 'O'), ('저', 'XPN', 'B_LC'), ('폭포', 'NNG', 'I'), ('중간', 'NNG', 'O'), ('으로', 'JKB', 'O'), ('나서', 'VV', 'O'), ('ㄴ다', 'EC', 'O'), ('.', 'SF', 'O')]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B_LC', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'B_LC', 'I', 'O', 'B_LC', 'I

In [177]:
# 문장 하나 확인
# 각 단어의 features가 dictionary로 표현되고 이를 요소로 하는 리스트
X[0]

[{'bias': 1.0,
  'word': '한편',
  'word.isdigit()': False,
  'postag': 'NNG',
  'postag[:2]': 'NN',
  'BOS': True,
  '+1:word': ',',
  '+1:word.isdigit()': False,
  '+1:postag': 'SP',
  '+1:postag[:2]': 'SP'},
 {'bias': 1.0,
  'word': ',',
  'word.isdigit()': False,
  'postag': 'SP',
  'postag[:2]': 'SP',
  '-1:word': '한편',
  '-1:word.isdigit()': False,
  '-1:postag': 'NNG',
  '-1:postag[:2]': 'NN',
  '+1:word': 'AFC',
  '+1:word.isdigit()': False,
  '+1:postag': 'SL',
  '+1:postag[:2]': 'SL'},
 {'bias': 1.0,
  'word': 'AFC',
  'word.isdigit()': False,
  'postag': 'SL',
  'postag[:2]': 'SL',
  '-1:word': ',',
  '-1:word.isdigit()': False,
  '-1:postag': 'SP',
  '-1:postag[:2]': 'SP',
  '+1:word': '챔피언스',
  '+1:word.isdigit()': False,
  '+1:postag': 'NNG',
  '+1:postag[:2]': 'NN'},
 {'bias': 1.0,
  'word': '챔피언스',
  'word.isdigit()': False,
  'postag': 'NNG',
  'postag[:2]': 'NN',
  '-1:word': 'AFC',
  '-1:word.isdigit()': False,
  '-1:postag': 'SL',
  '-1:postag[:2]': 'SL',
  '+1:word':

# Conditional Random Fields 적용 

In [178]:
from sklearn_crfsuite import CRF

# Graident Descent 대신 Limited-memory Broyden–Fletcher–Goldfarb–Shanno (LBFGS) 사용
crf = CRF(algorithm='lbfgs',
          c1=0.1, # overfitting을 방지하기 위한 L1 regularization(정규화) 가중치
          c2=0.1, # overfitting을 방지하기 위한 L2 regularization 가중치
          max_iterations=100,
          all_possible_transitions=False)

In [179]:
from sklearn.model_selection import cross_val_predict
from sklearn_crfsuite.metrics import flat_classification_report

In [180]:
# Cross validation 을 적용하여 CRF 학습 
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)

In [181]:
pred[0]

['O',
 'O',
 'B_OG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B_OG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [182]:
# predicted labels와 ground truth labels 비교
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

              precision    recall  f1-score   support

        B_DT       0.95      0.82      0.88      2135
        B_LC       0.71      0.62      0.66      2125
        B_OG       0.76      0.48      0.59      3267
        B_PS       0.68      0.75      0.71      2347
        B_TI       0.94      0.74      0.83       332
           I       0.84      0.67      0.75      7130
           O       0.96      0.99      0.97    103673

    accuracy                           0.94    121009
   macro avg       0.83      0.72      0.77    121009
weighted avg       0.94      0.94      0.94    121009



In [183]:
print(len(X), len(X)*0.7)
print(type(X))

3555 2488.5
<class 'list'>


In [184]:
# Training
crf.fit(X[:2488], y[:2488])

CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [185]:
# prediction
y_test_pred = crf.predict(X[2488:])

In [186]:
y[2488]
y_test_pred[0]

['B_PS',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B_DT',
 'I',
 'O',
 'O',
 'O',
 'B_DT',
 'I',
 'I',
 'I',
 'O',
 'B_DT',
 'I',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B_LC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B_DT',
 'I',
 'O',
 'O',
 'O',
 'O']

In [187]:
# predicted labels와 ground truth labels 비교
report = flat_classification_report(y_pred=y_test_pred, y_true=y[2488:])
print(report)

              precision    recall  f1-score   support

        B_DT       0.94      0.75      0.83       648
        B_LC       0.64      0.49      0.55       627
        B_OG       0.78      0.52      0.62       993
        B_PS       0.60      0.78      0.68       603
        B_TI       0.92      0.64      0.76       123
           I       0.82      0.69      0.75      2172
           O       0.96      0.99      0.97     27203

    accuracy                           0.93     32369
   macro avg       0.81      0.69      0.74     32369
weighted avg       0.93      0.93      0.93     32369



In [188]:
# 태그 간의 transition(전이) probabilities, 태그 별 예측에 중요한 features
import eli5
eli5.show_weights(crf, top=30)

In [210]:
# 위의 결과를 보면 너무 많은 features들이 사용되었다. 
# 드물게 발생하는 features의 영향력을 줄일 수 있는 L1 regularization의 가중치를 높여서 학습

crf = CRF(algorithm='lbfgs',
          c1=5,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [211]:
# cross validation
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)

In [212]:
# report
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

              precision    recall  f1-score   support

        B_DT       0.93      0.70      0.80      2135
        B_LC       0.55      0.50      0.53      2125
        B_OG       0.66      0.25      0.36      3267
        B_PS       0.50      0.63      0.56      2347
        B_TI       0.85      0.43      0.57       332
           I       0.82      0.51      0.63      7130
           O       0.94      0.99      0.97    103673

    accuracy                           0.92    121009
   macro avg       0.75      0.57      0.63    121009
weighted avg       0.91      0.92      0.91    121009



In [213]:
# Training
crf.fit(X[:2488], y[:2488])

CRF(algorithm='lbfgs', all_possible_transitions=False, c1=5, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [214]:
# prediction
y_test_pred = crf.predict(X[2488:])

In [215]:
# predicted labels와 ground truth labels 비교
report = flat_classification_report(y_pred=y_test_pred, y_true=y[2488:])
print(report)

              precision    recall  f1-score   support

        B_DT       0.91      0.64      0.75       648
        B_LC       0.47      0.37      0.41       627
        B_OG       0.69      0.23      0.34       993
        B_PS       0.43      0.74      0.54       603
        B_TI       0.87      0.21      0.34       123
           I       0.82      0.53      0.65      2172
           O       0.94      0.99      0.96     27203

    accuracy                           0.91     32369
   macro avg       0.73      0.53      0.57     32369
weighted avg       0.90      0.91      0.90     32369



In [216]:
# 태그 간의 transition(전이) probabilities, 태그 별 예측에 중요한 features
eli5.show_weights(crf, top=30)